In [1]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

#

# Code:

In [2]:
# Poplar 1x4". Cut
BlockHeight = 68.0  # [mm]
BlockLength = 143  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


68.0 x 143 x 19.0


In [3]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [4]:
if "Alarm" in cnc.status:
    cnc.home()

In [5]:
cnc.init()

True

# Generate Program


In [6]:
def dot(X=10, Y=10, Z=0, power=1, dwell=1, **kwargs):
    program = GCode.GCode(**kwargs)
    
    program.G0(X=X, Y=Y, Z=Z)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.

    return program

In [13]:
spacings=[0.05, 0.1, 0.2, 0.5, 1, 2]
len(spacings)

6

In [14]:
powers = [50, 75, 100, 150, 200, 255]
len(powers)

6

In [18]:
dwells = [0.1, 0.5]
len(dwells)

2

In [19]:
margin = 5

In [20]:
row_spacing=2

In [21]:
cnc.init()

True

In [61]:
loop0=powers
loop1=dwells
loop2=spacings

In [58]:
test_len=10 # [mm]
for idx0, power in enumerate(loop0):
    X0_next=margin # [mm]
    Y = margin + idx0*5
    for idx1, dwells in enumerate(loop1):
        for idx2, spacing in enumerate(loop2):
            X0=X0_next+(idx2*(10))
            Xf=X0+10
            XX=np.arange(X0, Xf, spacing)
            print("{} {} {} {} {}".format(X0, Xf, Y, power))
        X0_next = Xf+5

In [71]:
margins = 10 # [mm]
test_len=10 # [mm]
for idx0, power in enumerate(powers):
    X0_next=margin # [mm]
    Y = margin + idx0*5 # [mm]
    for idx1, dwell in enumerate(dwells):
        for idx2, spacing in enumerate(spacings):
            X0=X0_next+(idx2*(10)) #[mm]
            Xf=X0+10 #[mm]
            XX=np.arange(X0, Xf, spacing)
            dump = (X0, Xf, Y, power, dwell, spacing)
            print(("{:5.0f} "*len(dump)).format(*dump))
        X0_next = Xf+5

    5    15     5    50     0     0 
   15    25     5    50     0     0 
   25    35     5    50     0     0 
   35    45     5    50     0     0 
   45    55     5    50     0     1 
   55    65     5    50     0     2 
   70    80     5    50     0     0 
   80    90     5    50     0     0 
   90   100     5    50     0     0 
  100   110     5    50     0     0 
  110   120     5    50     0     1 
  120   130     5    50     0     2 
    5    15    10    75     0     0 
   15    25    10    75     0     0 
   25    35    10    75     0     0 
   35    45    10    75     0     0 
   45    55    10    75     0     1 
   55    65    10    75     0     2 
   70    80    10    75     0     0 
   80    90    10    75     0     0 
   90   100    10    75     0     0 
  100   110    10    75     0     0 
  110   120    10    75     0     1 
  120   130    10    75     0     2 
    5    15    15   100     0     0 
   15    25    15   100     0     0 
   25    35    15   100     0     0 
 

In [82]:
cnc.x_steps_mm

800.0

In [83]:
1/cnc.x_steps_mm

0.00125

In [86]:
1/cnc.x_steps_mm*16

0.02

In [91]:
1/cnc.x_steps_mm*2048

2.56

In [93]:
spacings=[1/cnc.x_steps_mm*8, 1/cnc.x_steps_mm*16, 1/cnc.x_steps_mm*64, 1/cnc.x_steps_mm*128, 1/cnc.x_steps_mm*1024, 1/cnc.x_steps_mm*1024, 1/cnc.x_steps_mm*2048]
spacings

[0.01, 0.02, 0.08, 0.16, 1.28, 1.28, 2.56]

In [98]:
powers = [50, 75, 100, 150, 200, 255]
powers

[50, 75, 100, 150, 200, 255]

In [110]:
dwells = [0.1, 0.5, 1]
dwells

[0.1, 0.5, 1]

In [116]:
cnc.init()
margins = 10 # [mm]
test_len=5 # [mm]
for idx0, power in enumerate(powers):
    X0_next=margin # [mm]
    Y = margin + idx0*5 # [mm]
    assert(Y<BlockHeight)
    for idx1, dwell in enumerate(dwells):
        for idx2, spacing in enumerate(spacings):
            X0=X0_next+(idx2*(test_len)) #[mm]
            Xf=X0+test_len #[mm]
            assert(Xf<BlockLength)
            XX=np.arange(X0, Xf, spacing)
            print("Laserin({}:{}, {}, {}, {})".format(X0, Xf, Y, power, dwell), end="")
            for X in XX:
                program = dot(X=X, Y=Y, Z=-14, power=power, dwell=dwell)
                cnc.run(program)
            print("... Done")
        X0_next = Xf+test_len

Laserin(5:10, 5, 50, 0.1)... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
...

... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
.

... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
.

... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
.

... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
.

...

... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
Laserin(50:55, 25, 200, 0.5)... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done


... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
Laserin(90:95, 25, 200, 1)... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
..

... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
.

... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
.

... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
... Done
.